In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [42]:
data = pd.read_csv("https://github.com/YBI-Foundation/Dataset/raw/main/IRIS.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [44]:
data.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [45]:
data.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

#Train Test Split

In [46]:
x,tx = train_test_split(data, random_state=2529)

#Data Preprocessing

In [47]:
col = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

att = x[col].values.tolist()
y = x[['species']].values.tolist()
Y = np.zeros(len(y))

def classToVal(s):
  if s==species[0]:
    return 0.1
  if s==species[1]:
    return 0.5
  if s==species[2]:
    return 0.9

for i,v in enumerate(y):
  a=classToVal(v[0])
  Y[i]=np.log(a/(1-a))

for v in att:
  v.append(1)

X = np.array(att)

#Least Square Approximator

In [48]:
xty = np.transpose(X) @ Y
xtx = np.transpose(X) @ X
W = np.linalg.inv(xtx) @ xty

def lsa(v):
  lny = np.dot(W,v)
  return np.exp(lny)/(1+np.exp(lny))

def lrc(v):
  d=np.zeros(3)
  t = lsa(v)
  d[0] = np.sqrt((t-0.1)*(t-0.1))
  d[1] = np.sqrt((t-0.5)*(t-0.5))
  d[2] = np.sqrt((t-0.9)*(t-0.9))

  grp = np.argmin(d)
  return species[grp]

#Testing The Model

In [49]:
N = len(tx)
tvf = tx[col].values.tolist()
for v in tvf:
  v.append(1)
tv = np.array(tvf)
tar = tx[['species']].values.tolist()
conf = np.zeros((3,3))

for i,v in enumerate(tv):
  prd = lrc(v)
  if prd==species[0]:
    if tar[i][0]==species[0]:
      conf[0][0]+=1
    if tar[i][0]==species[1]:
      conf[0][1]+=1
    if tar[i][0]==species[2]:
      conf[0][2]+=1
  elif prd==species[1]:
    if tar[i][0]==species[0]:
      conf[1][0]+=1
    if tar[i][0]==species[1]:
      conf[1][1]+=1
    if tar[i][0]==species[2]:
      conf[1][2]+=1
  elif prd==species[2]:
    if tar[i][0]==species[0]:
      conf[2][0]+=1
    if tar[i][0]==species[1]:
      conf[2][1]+=1
    if tar[i][0]==species[2]:
      conf[2][2]+=1

print(f"Out of {conf[0][0]+conf[1][0]+conf[2][0]} classified as Iris-setosa {conf[0][0]} were correctly predicted")
print(f"Out of {conf[0][1]+conf[1][1]+conf[2][1]} classified as Iris-versicolor {conf[1][1]} were correctly predicted")
print(f"Out of {conf[0][2]+conf[1][2]+conf[2][2]} classified as Iris-virginica {conf[2][2]} were correctly predicted")
n = conf[0][0]+conf[1][1]+conf[2][2]
print(f"{n} were correctly classified out of {N}")
print(f"Accuracy:{n*100/N}")

Out of 13.0 classified as Iris-setosa 13.0 were correctly predicted
Out of 6.0 classified as Iris-versicolor 6.0 were correctly predicted
Out of 19.0 classified as Iris-virginica 19.0 were correctly predicted
38.0 were correctly classified out of 38
Accuracy:100.0
